In [1]:
import sys
sys.path.append('../StableMotifs/')

import stablemotifs
import biolqm
import ginsim
import StableMotifs as sm
import PyBoolNet
import boolsim
import timeit
import timeout_decorator
from timeout_decorator import TimeoutError
import pandas as pd
import glob
from colomoto_jupyter import tabulate
import numpy as np



Bad key "text.kerning_factor" on line 4 in
/home/david/anaconda2/envs/python_3_env/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


## Generating Random Boolean Networks for the benchmarks

In [2]:

rbn_models_path='rbn_models_for_benchmark/'
generate_new_models=True
#this part populates the folder given in rbn_models_path with random boolean networks
#WARNING! the folder already has models in it from previous runs, those will be ran too.
#for a completely fresh run, empty the folder, or specify the model files considered explicitly below in the model_set
if generate_new_models:
    K=2
    p_bias=0.5
    N_ensemble=10 # the number of networks of the same size
    seed=1991
    for N in range(10,40,10): #the range of sizes
        rbn_ensemble_rules=sm.RandomBooleanNetworks.Random_Boolean_Network_Ensemble_Kauffman(N,K,p_bias,N_ensemble,seed=seed,write_Boolean_network=False)
        for i in range(N_ensemble):
            with open(rbn_models_path+'rbn_N_%d_ens_%d.booleannet'%(N,i),'w') as f:
                f.write(rbn_ensemble_rules[i])
        

In [3]:

#run_ens_size is the number of times each method is repeatedly run IN THE SAME ITERATION. 
#this means that the timeout has to be increased proporionally to account for all runs
#In the final output the minimum (best) runtime is saved out of all in the ensemble 
run_ens_size=1 

#timeout limit in seconds for each method (all runs within the run_ens_size range). 
#If the timeout limit is reached the output will be -1
timeout_limit=60


#name of methods to be exluded from the run. This can be set manually if we know 
#that some methods are not relevant or take too much time
exclude_from_run=set(['bioLQM',
                      'PyBoolNet_Asp',
                  #    'PyBoolNet',
                      'Pint',
                      #'sm_jgtz',
                      #'stable_motifs_new',
                      #'boolsim',
                      #'CABEAN'
                    ])
#exclude_from_run=set(['bioLQM','PyBoolNet_Asp','PyBoolNet','Pint']) 
#uncomment the line abovr if you want only methods that look for complex attractors

#the excel file in which the dataframe summarizing the results will be saved.
#if it already exists new results will be added to it
output_file_name='timing_results_RBN.xlsx'

In [4]:
import stablemotifs
import biolqm
import ginsim
import StableMotifs as sm
import PyBoolNet
import boolsim
import cabean
import pypint
from colomoto_jupyter import tabulate

#grabbing all the .booleannet models from the folder
model_set=glob.glob(rbn_models_path+'*.booleannet')
#model_set=['PSO.txt', 'phase_switch.txt','myeloid.txt']

for model_file_name in model_set:
    print('Running model %s'%model_file_name)
model_file_name='%s'%model_file_name
print(model_file_name)
with open(model_file_name) as f:
    rules=f.read().replace('#BOOLEAN RULES','').strip()
biolqm_model = biolqm.load(model_file_name, "booleannet")
pypint_model=biolqm.to_pint(biolqm_model)
print(biolqm_model)


Running model rbn_models_for_benchmark/rbn_N_10_ens_6.booleannet
Running model rbn_models_for_benchmark/rbn_N_10_ens_3.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_7.booleannet
Running model rbn_models_for_benchmark/rbn_N_30_ens_2.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_5.booleannet
Running model rbn_models_for_benchmark/rbn_N_30_ens_5.booleannet
Running model rbn_models_for_benchmark/rbn_N_30_ens_9.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_8.booleannet
Running model rbn_models_for_benchmark/rbn_N_10_ens_1.booleannet
Running model rbn_models_for_benchmark/rbn_N_30_ens_8.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_3.booleannet
Running model rbn_models_for_benchmark/rbn_N_10_ens_5.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_4.booleannet
Running model rbn_models_for_benchmark/rbn_N_20_ens_6.booleannet
Running model rbn_models_for_benchmark/rbn_N_10_ens_9.booleannet
Running model rbn_models_

In [5]:
@timeout_decorator.timeout(timeout_limit)
def run_timed_test(method_runstring, setup, repeat):

    return timeit.repeat(method_runstring,setup=setup, repeat=repeat, number=1)

try:
    result_df=pd.read_excel(output_file_name)
    result_df=result_df.reset_index(drop=True)
    already_ran_=result_df.model
except FileNotFoundError:
    result_df=pd.DataFrame()
    already_ran_=[]


In [6]:
#grabbing all the .booleannet models from the folder
model_set=glob.glob(rbn_models_path+'*.booleannet')
#model_set=['PSO.txt', 'phase_switch.txt','myeloid.txt']


In [7]:
list(already_ran_)

['rbn_N_10_ens_0',
 'rbn_N_10_ens_1',
 'rbn_N_10_ens_2',
 'rbn_N_10_ens_3',
 'rbn_N_10_ens_4',
 'rbn_N_10_ens_5',
 'rbn_N_10_ens_6',
 'rbn_N_10_ens_7',
 'rbn_N_10_ens_8',
 'rbn_N_10_ens_9',
 'rbn_N_20_ens_0']

In [8]:
for model_file_name in sorted(model_set):
    if model_file_name.split('/')[-1].split('.')[0] in list(already_ran_):
        print(model_file_name + ' already in models ran, moving on to the next model')
        continue
    
    print('Running model %s'%model_file_name)
    
    base_setup_string='''import stablemotifs
import biolqm
import json
import ginsim
import StableMotifs as sm
import PyBoolNet
import boolsim
import cabean
import pypint
from colomoto_jupyter import tabulate


model_file_name='%s'
with open(model_file_name) as f:
    rules=f.read().replace('#BOOLEAN RULES','').strip()
biolqm_model = biolqm.load(model_file_name, "booleannet")
pypint_model=biolqm.to_pint(biolqm_model)'''%model_file_name

    method_runstrings={'sm_jgtz': "a=stablemotifs.load(biolqm_model, quiet=True).attractors\ntabulate(a).to_excel('attr_output/attr_output_sm_jgtz_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'stable_motifs_new': "ar = sm.AttractorRepertoire.from_primes(primes, max_simulate_size=max_simulate_size)\nex.attractor_dataframe(ar).to_excel('attr_output/attr_output_stable_motifs_new_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')\nprint(ar.fewest_attractors,ar.most_attractors)",
                   
                   
 'PyBoolNet': 'd=PyBoolNet.Attractors.compute_json(primes, "asynchronous")\nwith open(\'attr_output/attr_output_PyBoolNet_\'+model_file_name.split(\'.\')[0].split(\'/\')[-1]+\'.json\',\'w\') as fp:\n\tjson.dump(d,fp)',
                   
                   
 'boolsim': "a=boolsim.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_boolsim_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'bioLQM': "a=biolqm.fixpoints(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_bioLQM_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'PyBoolNet_Asp': "a=PyBoolNet.AspSolver.steady_states(primes)\ntabulate(a).to_excel('attr_output/attr_output_PyBoolNet_Asp_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'CABEAN': "a=cabean.attractors(biolqm_model)\ntabulate(a).to_excel('attr_output/attr_output_CABEAN_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')",
                   
                   
 'Pint': "a=pypint.fixpoints(pypint_model)\ntabulate(a).to_excel('attr_output/attr_output_Pint_'+model_file_name.split('.')[0].split('/')[-1]+'.xlsx')", 
                      
  }

    method_setups={'sm_jgtz':base_setup_string,

    'stable_motifs_new':base_setup_string+'''
max_simulate_size=20
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
import StableMotifs.Export as ex
''',

    'PyBoolNet':base_setup_string+'''
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
''',

    'boolsim':base_setup_string,

    'bioLQM':base_setup_string,

    'PyBoolNet_Asp':base_setup_string+'''
rules_pbn = sm.Format.booleannet2bnet(rules)
primes = PyBoolNet.FileExchange.bnet2primes(rules_pbn)
''',

    'CABEAN':base_setup_string,

    'Pint':base_setup_string}

    method_runtimes={i:[] for i in method_runstrings}
    
    
    for method_name in set(method_runstrings)-exclude_from_run:
        print('method:',method_name)
        try:
            method_runtimes[method_name]=run_timed_test(method_runstrings[method_name], setup=method_setups[method_name], repeat=run_ens_size)
        except Exception as e:
            print(str(e))
            method_runtimes[method_name].append(np.nan)
    runtime_minimums={k:min(method_runtimes[k]) for k in method_runtimes.keys()-exclude_from_run} 
    runtime_minimums['model']=model_file_name.split('.')[0].split('/')[-1]
    result_df=result_df.append([runtime_minimums],ignore_index=True)
    result_df=result_df.reset_index(drop=True)

    result_df.to_excel(output_file_name)

rbn_models_for_benchmark/rbn_N_10_ens_0.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_1.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_2.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_3.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_4.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_5.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_6.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_7.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_8.booleannet already in models ran, moving on to the next model
rbn_models_for_benchmark/rbn_N_10_ens_9.booleannet already in models ran, moving o

 Attractors.completeness(..) yes
 working on minimal trapspace 1/1: 010110000110111100001111110001
  Attractors.univocality(..) yes
  Attractors.faithfulness(..) yes
  Attractors.find_attractor_state_by_randomwalk_and_ctl(..)
method: boolsim
method: stable_motifs_new
1 1
method: CABEAN
Running model rbn_models_for_benchmark/rbn_N_30_ens_4.booleannet
method: sm_jgtz
method: PyBoolNet
Attractors.compute_json(..)
 Attractors.completeness(..) yes
 working on minimal trapspace 1/1: 0110010111111110100110-0-00010
  Attractors.univocality(..) yes
  Attractors.faithfulness(..) yes
  Attractors.find_attractor_state_by_randomwalk_and_ctl(..)
method: boolsim
method: stable_motifs_new
1 1
method: CABEAN
Running model rbn_models_for_benchmark/rbn_N_30_ens_5.booleannet
method: sm_jgtz
method: PyBoolNet
Attractors.compute_json(..)
 Attractors.completeness(..) yes
 working on minimal trapspace 1/2: 001011000001010111111001011100
  Attractors.univocality(..) yes
  Attractors.faithfulness(..) yes
  Attr

In [9]:
result_df.to_excel(output_file_name)